In [4]:
!pip install skyfield
from skyfield.api import load, wgs84
import pytz
import pylab as pl
import matplotlib.pyplot as plt
ts = load.timescale()
eastern = pytz.timezone('US/Eastern')
raleigh = wgs84.latlon(+35.78, -78.64, elevation_m=100)


[space station orbital elements (NORAD)](http://celestrak.com/NORAD/elements/stations.txt)


In [5]:
stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

Loaded 69 satellites


In [6]:
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['ISS (ZARYA)']
print(satellite)
print(satellite.epoch.utc_jpl())
print(satellite)

ISS (ZARYA) catalog #25544 epoch 2021-01-30 12:01:55 UTC
A.D. 2021-Jan-30 12:01:55.1634 UTC
ISS (ZARYA) catalog #25544 epoch 2021-01-30 12:01:55 UTC


Where is the ISS? [NASA Spot the Station](https://spotthestation.nasa.gov/tracking_map.cfm)

In [7]:
t = ts.now()
geocentric = satellite.at(t)
subpoint = wgs84.subpoint(geocentric)
print(f'Latitude:  {subpoint.latitude.degrees:.2f}')
print(f'Longitude: {subpoint.longitude.degrees:.2f}')
print(f"Elevation {int(subpoint.elevation.km)} (km) {int(subpoint.elevation.km*0.621371)} (mi)")
print(f"https://earth.google.com/web/search/{subpoint.latitude.degrees:.2f},{subpoint.longitude.degrees:.2f}")

Latitude:  51.78
Longitude: 25.69
Elevation 426 (km) 265 (mi)
https://earth.google.com/web/search/51.78,25.69


In [8]:
difference = satellite - raleigh
topocentric = difference.at(t)
alt, az, distance = topocentric.altaz()
print(f"alt {alt}, az {az} distance {int(distance.km)} (km) {int(distance.km*0.621371):,} (mi)")

alt -32deg 36' 16.6", az 39deg 32' 45.4" distance 7619 (km) 4,734 (mi)


ISS Passes [Heavens Above](https://www.heavens-above.com/main.aspx?lat=35.78&lng=-78.64&loc=Raleigh,+NC&alt=200.00000000000003)

In [9]:
raleigh = wgs84.latlon(+35.78, -78.64, elevation_m=100)
t0 = ts.utc(2021, 1, 31)
t1 = ts.utc(2021, 2, 5)
t, events = satellite.find_events(raleigh, t0, t1, altitude_degrees=10.0)
for ti, event in zip(t, events):
    name = ('rise above 10°', 'culminate', 'set below 10°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2021 Jan 31 01:43:20 rise above 10°
2021 Jan 31 01:46:34 culminate
2021 Jan 31 01:49:47 set below 10°
2021 Jan 31 03:20:56 rise above 10°
2021 Jan 31 03:23:07 culminate
2021 Jan 31 03:25:17 set below 10°
2021 Jan 31 18:24:25 rise above 10°
2021 Jan 31 18:27:44 culminate
2021 Jan 31 18:31:05 set below 10°
2021 Jan 31 20:02:44 rise above 10°
2021 Jan 31 20:04:51 culminate
2021 Jan 31 20:07:00 set below 10°
2021 Feb 01 00:56:12 rise above 10°
2021 Feb 01 00:59:05 culminate
2021 Feb 01 01:01:59 set below 10°
2021 Feb 01 02:32:54 rise above 10°
2021 Feb 01 02:35:53 culminate
2021 Feb 01 02:38:52 set below 10°
2021 Feb 01 17:37:20 rise above 10°
2021 Feb 01 17:40:25 culminate
2021 Feb 01 17:43:32 set below 10°
2021 Feb 01 19:14:34 rise above 10°
2021 Feb 01 19:17:18 culminate
2021 Feb 01 19:20:02 set below 10°
2021 Feb 02 00:09:10 rise above 10°
2021 Feb 02 00:11:32 culminate
2021 Feb 02 00:13:54 set below 10°
2021 Feb 02 01:45:16 rise above 10°
2021 Feb 02 01:48:35 culminate
2021 Feb 02 01:

In [11]:
eph = load('de421.bsp')
sun, earth = eph['sun'], eph['earth']
horizon=10
def sun_pos(t, topopos):
  obs = earth + raleigh
  astrometric = obs.at(t).observe(sun)
  apparent = obs.at(t).observe(sun).apparent()
  alt, az, dist = apparent.altaz()
  return alt, az, dist

In [12]:
for ti, event in zip(t, events):
    name = (f'rise above {horizon}°', 'highest point', f'set below {horizon}°')[event]
    sunalt, sunaz, sundist = sun_pos(ti, raleigh)
    if -12 <= sunalt.degrees <= -6:
      print(ti.utc_jpl(), name)

A.D. 2021-Feb-02 23:22:22.5278 UTC rise above 10°
A.D. 2021-Feb-02 23:23:55.4575 UTC highest point
A.D. 2021-Feb-02 23:25:29.0701 UTC set below 10°
A.D. 2021-Feb-04 23:23:23.2923 UTC rise above 10°
A.D. 2021-Feb-04 23:26:17.6494 UTC highest point
A.D. 2021-Feb-04 23:29:11.5232 UTC set below 10°


In [13]:

horizon=10.0
t, events = satellite.find_events(raleigh, t0, t1, altitude_degrees=horizon)
for ti, event in zip(t, events):
    name = (f'rise above {horizon}°', 'highest point', f'set below {horizon}°')[event]
    sunalt, sunaz, sundist = sun_pos(ti, raleigh)
    if -12 <= sunalt.degrees <= -6:
      print(ti.astimezone(eastern).strftime('%a %b %d %-I:%M:%S %p'), name)
      if event ==2:
        print()
        # plot_sky(passes[1], self.timezoneobj, t, alt, az)

Tue Feb 02 6:22:22 PM rise above 10.0°
Tue Feb 02 6:23:55 PM highest point
Tue Feb 02 6:25:29 PM set below 10.0°

Thu Feb 04 6:23:23 PM rise above 10.0°
Thu Feb 04 6:26:17 PM highest point
Thu Feb 04 6:29:11 PM set below 10.0°

